In [3]:
import sys
sys.path.append("../")

import subprocess
import datetime
from joblib import Parallel, delayed

import json
import os

import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna

from run import *

In [6]:
start_case = 0
case_num = 500

cases = [
    (f"tools/in/{seed:04}.txt", f"tools/out/{seed:04}.txt")
    for seed in range(start_case, start_case + case_num)
]

study_name = "optuna-anneal-temp-04"


def objective(trial: optuna.trial.Trial) -> float:
    start_temp_d1 = trial.suggest_float("start_temp_d1", low=1e0, high=1e4)
    end_temp_d1 = trial.suggest_float(
        "end_temp_d1", low=1e-3, high=min(start_temp_d1 - 1, 1e1)
    )
    start_temp_d2 = trial.suggest_float("start_temp_d2", low=1e0, high=1e4)
    end_temp_d2 = trial.suggest_float(
        "end_temp_d2", low=1e-3, high=min(start_temp_d2 - 1, 1e1)
    )

    args = " ".join(
        map(
            str,
            [
                start_temp_d1,
                end_temp_d1,
                start_temp_d2,
                end_temp_d2,
            ],
        )
    )

    solver_path = "./target/release/ahc031"
    solver_cmd = f"{solver_path} {args}"
    solver_version = f"{study_name}-{args}"
    runner = Runner(
        Input,
        Result,
        solver_cmd=solver_cmd,
        solver_version=solver_version,
        database_csv="../log/database.csv",
        input_csv="../log/input.csv",
        log_file="../log/a.log",
        is_maximize=False,
        verbose=0,
    )
    runner.run(cases=cases, verbose=0)
    score_df = runner.evalute_relative_score(ignore_solver_prefix=f"{study_name}-")
    return score_df.relative_score.mean()


subprocess.run(
    "cargo build --features local --release --manifest-path=../Cargo.toml", shell=True
)

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///ahc031.db",
    study_name=study_name,
    load_if_exists=True,
)
print("start optimize")
# study.enqueue_trial(
#     {
#         "start_temp_d1": 1e1,
#         "end_temp_d1": 1e-1,
#         "start_temp_d2": 1e1,
#         "end_temp_d2": 1e-1,
#     }
# )
study.optimize(objective, n_trials=100, show_progress_bar=True)
study.best_params

    Finished release [optimized] target(s) in 0.04s
[I 2024-03-31 17:32:37,021] Using an existing study with name 'optuna-anneal-temp-04' instead of creating a new one.


start optimize


/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-03-31 17:34:40,881] Trial 2 finished with value: 0.8296531653862788 and parameters: {'start_temp_d1': 4201.714270563002, 'end_temp_d1': 6.770375685286039, 'start_temp_d2': 7917.7994404627125, 'end_temp_d2': 8.03467058744462}. Best is trial 2 with value: 0.8296531653862788.
[I 2024-03-31 17:36:44,175] Trial 3 finished with value: 0.8252166467048568 and parameters: {'start_temp_d1': 484.87492801014133, 'end_temp_d1': 6.0856394803397675, 'start_temp_d2': 1326.122635506648, 'end_temp_d2': 0.3749754845645935}. Best is trial 2 with value: 0.8296531653862788.
[I 2024-03-31 17:38:47,451] Trial 4 finished with value: 0.8230052470512614 and parameters: {'start_temp_d1': 4901.3861889697255, 'end_temp_d1': 1.6306688544053873, 'start_temp_d2': 2646.923342707526, 'end_temp_d2': 0.7182272778208936}. Best is trial 2 with value: 0.8296531653862788.
[I 2024-03-31 17:40:50,645] Trial 5 finished with value: 0.8272116241195242 and parameters: {'start_temp_d1': 6646.0970125231, 'end_temp_d1': 0.3228

KeyboardInterrupt: 